# Projet 3 : Concevez une application au service de la santé publique #

### Import du nécessaire ###

In [1]:
import numpy as np
import annexe as an
# import plotly.graph_objs as go
import re
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', None)
FICHIER ='F:/cour/OC/projet3/en.openfoodfacts.org.products.csv'

%load_ext autoreload
%autoreload 2

## Observations


Observons quelques lignes de notre fichier

In [ ]:
food_dataframe = pd.read_csv(FICHIER,sep='\t',low_memory=False)
display(food_dataframe.sample(10,random_state = 148625))

Observons la liste de colonnes

In [3]:
for column in dataframe.columns:
    print(column)

code
url
creator
created_t
created_datetime
last_modified_t
last_modified_datetime
product_name
abbreviated_product_name
generic_name
quantity
packaging
packaging_tags
packaging_text
brands
brands_tags
categories
categories_tags
categories_en
origins
origins_tags
origins_en
manufacturing_places
manufacturing_places_tags
labels
labels_tags
labels_en
emb_codes
emb_codes_tags
first_packaging_code_geo
cities
cities_tags
purchase_places
stores
countries
countries_tags
countries_en
ingredients_text
allergens
allergens_en
traces
traces_tags
traces_en
serving_size
serving_quantity
no_nutriments
additives_n
additives
additives_tags
additives_en
ingredients_from_palm_oil_n
ingredients_from_palm_oil
ingredients_from_palm_oil_tags
ingredients_that_may_be_from_palm_oil_n
ingredients_that_may_be_from_palm_oil
ingredients_that_may_be_from_palm_oil_tags
nutriscore_score
nutriscore_grade
nova_group
pnns_groups_1
pnns_groups_2
states
states_tags
states_en
brand_owner
ecoscore_score_fr
ecoscore_grade_fr


### Elements pertinant pour le sport
50% glucides : bon glucide **carbohydrates** on un faible indice glycémique **glycemic-index_100g
** (pain complet, pates de blé dur, légumineuse)
20% lipides (après l'effort pk pas, avant non) acide gras omégas 3 **'alpha-linolenic','eicosapentaenoic-acid','docosahexaenoic-acid' omega-3-fat** et omégas 6 (renforce coeur et circulation) (poisson eau froide saumon maquereau, huile végétal
15% proteines source précieuse de protéines : oeuf, viande maigre, produit laitier et legumineuses
3h entre le gros repas et sport (noix , morceau de choco avant le sport), blanc d'oeuf 2h d'interval, boire du lait écrémé
boire important pour l'acheminement des nutriment (riche en sodium), eau gazeuse plus jus de fruit apres l'effort(1 pour 1)
avant entrainnement: glucides(pates, pain complet), oeuf brouillées,fruit yaourt,pauvre en fibre
pendant celui-ci:eau, boisson isotonique,barre de muesli
apres:jus de pomme petillant avec une pincée de sel,pas de graisse,glucides, protéines de ahute qualité,produit laitier


Activité de moins de 30 min : Il n’est pas nécessaire de boire pendant l’effort
Activité de 30 à 60 min : il faut boire de 125 à 250 ml de liquide à toutes les 15 à 20 minutes.
Activité de 60 min, il ne faut pas attendre d’avoir soif pour boire de l’eau
Activité durant 1h à 3h : il est préférable de choisir une boisson contenant du sucre
Activité de plus de 3 h : préférez une boisson contenant du sucre et un peu de sel (pour le sodium). À noter que le sodium et le potassium dans la boisson de réhydratation ne sont pas indispensables si l’activité physique dure moins de 3 h à 4 h, à moins que la sudation engendrée par l’activité physique ne soit excessive.

Dataframe apres avoir garder que les colonnes pertinantes

In [2]:
dataframe_purify = an.open_csv_usefull_column()
# display(dataframe_purify.sample(10,random_state = 148625))

Voici quelques variables pertinentes pour un application orienté sport

In [3]:
trace_element_variables = ['sodium_100g','zinc_100g','copper_100g','potassium_100g','selenium_100g','manganese_100g','chromium_100g','iodine_100g']
acid_variables = ['omega-3-fat_100g','omega-6-fat_100g','omega-9-fat_100g']
vitamin_variables = ['vitamin-b1_100g','vitamin-b2_100g','vitamin-b6_100g']
sugar_variables = ['carbohydrates_100g','glycemic-index_100g','sugars_100g','-sucrose_100g','-glucose_100g','-fructose_100g','-lactose_100g','-maltose_100g','-maltodextrins_100g']
string_variables = ['pnns_groups_1','pnns_groups_2','nutriscore_score','nutriscore_grade','ecoscore_score_fr','ecoscore_grade_fr']

forme du dataframe avant et apres purification des colonnes avec valeurs abhérantes

In [4]:
print("shape ",dataframe_purify.shape)
regex = re.compile(".*_100g.*")
column_list = list(filter(regex.match, dataframe_purify.columns))
an.remove_little_quintille(dataframe_purify,column_list)
print("shape ",dataframe_purify.shape)

shape  (1760097, 152)
shape  (1611486, 150)


Visualisation de si l'algorythme a bien enlever les valeurs abhérentes

In [5]:
for elem in sugar_variables:
    print("\n\ncolumn : ",elem )
    display("max :",dataframe_purify[elem].max())
    display("min :",dataframe_purify[elem].min())
    display("moyenne :",dataframe_purify[elem].mean())
    display("mediane :",dataframe_purify[elem].median())
    display(dataframe_purify[elem].value_counts().head(10))#.sample(10,random_state = 148625))  



column :  carbohydrates_100g


'max :'

95.2

'min :'

0.0

...

'moyenne :'

2.8302933339393945

'mediane :'

0.12

0.12000     13
0.00012      5
1.50000      3
0.20000      2
14.80000     1
10.30000     1
0.22600      1
0.86000      1
3.40000      1
2.20000      1
Name: -maltodextrins_100g, dtype: int64

Affichage des variable string avant et apres traitement pour enlever les majuscule et les tiret

In [31]:
for elem in string_variables:
    display(dataframe_purify[elem].value_counts())


unknown                    863447
sugary snacks               97565
fish meat eggs              72814
milk and dairy products     70808
cereals and potatoes        64683
beverages                   56340
composite foods             47693
fruits and vegetables       40339
fat and sauces              34015
salty snacks                17640
Name: pnns_groups_1, dtype: int64

unknown                             863447
biscuits and cakes                   51602
one dish meals                       37231
sweets                               36255
dressings and sauces                 31718
cereals                              29722
milk and yogurt                      29524
cheese                               28698
processed meat                       24164
meat                                 23829
fish and seafood                     21095
vegetables                           21052
bread                                19683
sweetened beverages                  18722
alcoholic beverages                  15185
fruits                               13338
appetizers                            8943
unsweetened beverages                 8622
fruit juices                          8496
salty and fatty products              7492
artificially sweetened beverages      7239
pizza pies and quiche                 6864
breakfast cereals                     6734
ice cream  

 0.0     29274
 2.0     24657
 3.0     23944
 1.0     23662
-1.0     21752
 4.0     21290
 11.0    20648
 13.0    19874
 12.0    19646
 14.0    19471
 15.0    18180
 5.0     17898
 16.0    17077
-4.0     16492
-2.0     14771
-3.0     13609
 17.0    12548
 18.0    11974
 6.0     11848
 19.0    10867
 10.0    10768
 7.0      9974
 20.0     9517
 8.0      9303
-5.0      9081
 9.0      8918
 21.0     8337
-6.0      6994
 22.0     6641
 23.0     6493
 24.0     6270
 25.0     5150
 26.0     4787
Name: nutriscore_score, dtype: int64

d    133788
c    107277
a     83719
b     74664
e     72267
Name: nutriscore_grade, dtype: int64

 39.0     25474
 43.0     11263
 29.0     10047
 36.0      9343
 67.0      7871
 55.0      7095
 41.0      6998
 34.0      6876
 9.0       6876
 82.0      6766
 35.0      6647
 33.0      6558
 45.0      5946
 76.0      5805
 42.0      5782
 79.0      5704
 18.0      5662
 60.0      5577
 85.0      5531
 80.0      5455
 59.0      5265
 63.0      5186
 65.0      4991
 81.0      4935
 22.0      4552
 54.0      4355
 75.0      4334
 83.0      4287
 64.0      4122
 19.0      4059
 72.0      4024
 77.0      3593
 50.0      3390
 21.0      3340
 78.0      3313
 24.0      2727
 51.0      2692
 73.0      2593
 74.0      2568
 52.0      2517
 26.0      2453
 31.0      2383
 84.0      2327
 20.0      2324
 23.0      2271
 70.0      2252
 66.0      2223
 30.0      2214
 27.0      2149
 53.0      2137
 44.0      2113
 12.0      2053
 91.0      2017
 56.0      2013
 69.0      1936
 6.0       1903
 28.0      1849
 32.0      1758
 49.0      1702
 10.0      1681
 71.0      1548
 68.0      1488
 62.0   

b    107798
d     96832
c     70455
e     33745
a     14215
Name: ecoscore_grade_fr, dtype: int64

In [8]:
an.text_regularisation(dataframe_purify,['pnns_groups_1','pnns_groups_2'])
for elem in string_variables:
    display(dataframe_purify[elem].value_counts())

unknown                    993387
sugary snacks              127984
fish meat eggs              85850
milk and dairy products     82248
cereals and potatoes        74082
beverages                   59611
composite foods             50634
fruits and vegetables       47474
fat and sauces              45195
salty snacks                28749
Name: pnns_groups_1, dtype: int64

unknown                             993387
biscuits and cakes                   57831
sweets                               53618
dressings and sauces                 40874
one dish meals                       39864
cheese                               39099
cereals                              34587
processed meat                       33414
milk and yogurt                      30337
meat                                 25512
vegetables                           24350
fish and seafood                     23019
bread                                20716
sweetened beverages                  20182
fruits                               15369
alcoholic beverages                  15293
salty and fatty products             12623
chocolate products                   11270
appetizers                           10406
unsweetened beverages                 9110
fruit juices                          8648
breakfast cereals                     8399
artificially sweetened beverages      8002
legumes    

 0.0     31266
 14.0    27908
 13.0    27227
 2.0     26687
 11.0    25740
 3.0     25609
 1.0     25454
 12.0    24449
 15.0    24351
-1.0     23312
 4.0     23038
 16.0    23029
 5.0     19388
 17.0    17770
-4.0     17747
 18.0    16458
-2.0     16093
 19.0    15218
-3.0     14855
 10.0    14254
 6.0     13640
 20.0    13603
 9.0     12232
 21.0    12000
 7.0     11984
 8.0     11859
-5.0     10387
 24.0    10116
 22.0     9665
 23.0     9416
 25.0     9107
-6.0      8280
 26.0     8082
 27.0     4783
-7.0      3327
Name: nutriscore_score, dtype: int64

d    181132
c    125437
e    106537
a     95019
b     80209
Name: nutriscore_grade, dtype: int64

 39.0     29960
 43.0     12823
 34.0     11482
 29.0     11007
 9.0      10518
 36.0      9660
 67.0      9479
 55.0      7844
 41.0      7778
 45.0      7416
 33.0      7405
 42.0      7340
 82.0      7295
 35.0      7236
 19.0      7167
 60.0      6971
 80.0      6678
 85.0      6427
 18.0      6257
 63.0      6094
 76.0      6005
 79.0      5978
 21.0      5747
 59.0      5576
 65.0      5266
 81.0      5225
 75.0      4942
 22.0      4773
 54.0      4650
 83.0      4609
 64.0      4434
 72.0      4267
 74.0      4116
 77.0      4032
 24.0      3786
 50.0      3566
 30.0      3517
 78.0      3473
 20.0      2914
 51.0      2870
 6.0       2807
 73.0      2772
 26.0      2726
 52.0      2692
 31.0      2628
 56.0      2594
 84.0      2577
 23.0      2465
 66.0      2452
 70.0      2350
-8.0       2342
 44.0      2337
 53.0      2272
 12.0      2271
 27.0      2255
 49.0      2220
 91.0      2153
 32.0      2135
 69.0      2090
 28.0      2002
-4.0       1949
 10.0      1844
 68.0   

b    121574
d    116792
c     81008
e     47874
a     15947
Name: ecoscore_grade_fr, dtype: int64

In [ ]:
matrice de correlation des variables numériques

In [5]:
correlation_dataframe = dataframe_purify.corr(method='pearson')
display(correlation_dataframe)

,serving_quantity,nutriscore_score,ecoscore_score_fr,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
serving_quantity,1.000000,-0.012933,-0.001175,-0.006137,0.003262,-0.002311,-0.254859,-0.001372,-0.004934,-1.0,NaN,NaN,NaN,NaN,-0.474752,NaN,-0.043591,-0.476956,NaN,NaN,-0.259410,NaN,-0.104526,-0.108998,-0.252209,-0.224866,-0.389775,-0.152020,-0.418802,-0.478949,-0.094719,NaN,NaN,-0.239391,0.211481,NaN,0.701292,-0.039296,NaN,NaN,0.066054,-0.076949,-0.001813,-0.001272,-0.366552,-0.266331,-0.320408,-0.074617,-0.065776,-0.076104,-0.221106,-0.423337,-0.001414,-0.084010,-0.065172,-0.001547,-0.559520,-0.037059,0.860233,-0.001344,-0.001344,0.014833,0.012578,-0.349697,-0.037023,-0.020418,-0.134350,0.044668,0.039029,-0.004513,-0.177904,-0.008107,-0.081789,-0.316442,-0.071453,-0.074065,-0.278520,-0.414379,-0.152969,-0.247140,-0.442993,-0.097733,-0.433316,-0.246489,-0.386097,-0.314835,-0.375000,-0.499318,-0.358305,-0.341378,-0.144422,-0.069234,-0.158684,-0.048878,-0.114539,-0.021848,0.029831,-0.369290,-0.019851,-0.364285,-0.022656,-0.060941,-0.173907,-0.012933,-0.591245,NaN,NaN,0.145363,-0.250012,-0.423522,-0.585353,1.000000
nutriscore_score,-0.012933,1.000000,-0.342295,0.599187,0.603813,0.605880,0.405349,0.602753,0.670336,NaN,NaN,1.0,NaN,1.0,-0.180600,NaN,-0.032071,0.209063,NaN,NaN,-0.819204,NaN,0.331431,0.335923,0.270178,0.038846,0.382889,0.344432,0.320218,0.386358,-0.168212,NaN,NaN,0.479700,-0.244260,NaN,NaN,1.000000,NaN,-1.0,0.043950,0.263491,0.252331,0.450808,0.558191,0.262175,0.230900,0.194185,0.495627,0.350162,-0.180447,-0.015434,-0.136307,-0.435910,-0.494749,0.109523,NaN,0.599257,0.499154,0.328893,0.328894,0.035767,-0.038363,0.468626,0.027508,0.004744,-0.069206,-0.132123,0.008571,-0.001015,0.110525,-0.005804,-0.082591,-0.038966,-0.000503,-0.065016,0.267828,0.028236,-0.074122,0.041675,0.372321,0.154203,0.028079,0.068729,0.017537,0.181130,0.277854,0.008306,0.388983,-0.177573,-0.118296,0.067476,0.045266,0.052803,-0.065554,-0.236578,-0.334213,-0.137253,-0.301431,0.590666,-0.226445,0.330240,0.029400,1.000000,1.000000,NaN,NaN,NaN,-0.283879,-0.543264,0.632270,NaN
ecoscore_score_fr,-0.001

variables qui ont une corrélation de 1

In [13]:
an.variables_correlation_over_parameter(correlation_dataframe,0.9)

serving_quantity carnitine_100g
nutriscore_score -capric-acid_100g
nutriscore_score -myristic-acid_100g
nutriscore_score -mead-acid_100g
nutriscore_score nutrition-score-fr_100g
nutriscore_score nutrition-score-uk_100g
ecoscore_score_fr -cerotic-acid_100g
ecoscore_score_fr silica_100g
energy-kj_100g energy-kcal_100g
energy-kj_100g energy_100g
energy-kj_100g -oleic-acid_100g
energy-kj_100g chloride_100g
energy-kcal_100g energy_100g
energy-kcal_100g -myristic-acid_100g
energy-kcal_100g -montanic-acid_100g
energy_100g -myristic-acid_100g
energy_100g -montanic-acid_100g
energy-from-fat_100g molybdenum_100g
energy-from-fat_100g phylloquinone_100g
fat_100g -capric-acid_100g
fat_100g -myristic-acid_100g
fat_100g -palmitic-acid_100g
fat_100g -nervonic-acid_100g
saturated-fat_100g -capric-acid_100g
saturated-fat_100g -myristic-acid_100g
saturated-fat_100g -palmitic-acid_100g
saturated-fat_100g -nervonic-acid_100g
saturated-fat_100g nutrition-score-uk_100g
-butyric-acid_100g carbohydrates_100g
-

In [8]:
dataframe_copy = dataframe_purify.copy()

In [9]:
dataframe_copy['iodine_100g'].iloc[52410]

nan

In [12]:
variable1 = '-mead-acid_100g'
variable2 = 'caffeine_100g'
compt = 0
for row in range(dataframe_purify.shape[0]):
    if not np.isnan(dataframe_purify[variable1].iloc[row]) and not np.isnan(dataframe_purify[variable2].iloc[row]):
        display(dataframe_copy[variable1].iloc[row])
        display(dataframe_copy[variable2].iloc[row])
        print(row)
    else:
        compt += 1
        if compt == 100000:
            print(100000)
            compt = 0

100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000


In [22]:
dataframe_purify.columns.tolist()

['code',
 'url',
 'product_name',
 'abbreviated_product_name',
 'quantity',
 'packaging',
 'packaging_tags',
 'packaging_text',
 'brands',
 'brands_tags',
 'categories',
 'categories_tags',
 'categories_en',
 'emb_codes',
 'emb_codes_tags',
 'first_packaging_code_geo',
 'purchase_places',
 'stores',
 'countries',
 'countries_tags',
 'countries_en',
 'ingredients_text',
 'allergens',
 'traces',
 'traces_tags',
 'traces_en',
 'serving_quantity',
 'nutriscore_score',
 'nutriscore_grade',
 'pnns_groups_1',
 'pnns_groups_2',
 'ecoscore_score_fr',
 'ecoscore_grade_fr',
 'main_category',
 'main_category_en',
 'image_url',
 'image_small_url',
 'image_ingredients_url',
 'image_ingredients_small_url',
 'image_nutrition_url',
 'image_nutrition_small_url',
 'energy-kj_100g',
 'energy-kcal_100g',
 'energy_100g',
 'energy-from-fat_100g',
 'fat_100g',
 'saturated-fat_100g',
 '-butyric-acid_100g',
 '-caproic-acid_100g',
 '-capric-acid_100g',
 '-lauric-acid_100g',
 '-myristic-acid_100g',
 '-palmitic-ac

In [52]:
dataframe_purify[dataframe_purify["sodium_100g"] > 6]


,code,url,product_name,abbreviated_product_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,emb_codes,emb_codes_tags,first_packaging_code_geo,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_quantity,no_nutriments,nutriscore_score,nutriscore_grade,pnns_groups_1,pnns_groups_2,ecoscore_score_fr,ecoscore_grade_fr,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
64,0000000005081,http://world-en.openfoodfacts.org/product/0000000005081/salade-verte,Salade verte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,https://static.openfoodfacts.org/images/products/000/000/000/5081/front_fr.4.400.jpg,https://static.openfoodfacts.org/images/products/000/000/000/5081/front_fr.4.200.jpg,https://static.openfoodfacts.org/images/products/000/000/000/5081/ingredients_fr.8.400.jpg,https://static.openfoodfacts.org/images/products/000/000/000/5081/ingredients_fr.8.200.jpg,NaN,NaN,NaN,16.0,67.0,NaN,0.3,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3,NaN,NaN,NaN,18.30000,7.320000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,0000000018371,http://world-en.openfoodfacts.org/product/0000000018371/real-salt-granular-redmond,Real Salt Granular,NaN,NaN,NaN,NaN,NaN,Redmond,redmond,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,United States,Ancient sea salt with natural trace minerals,NaN,NaN,NaN,NaN,NaN,1.4,NaN,NaN,NaN,unknown,unknown,NaN,NaN,

In [21]:
dataframe_purify[dataframe_purify["selenium_100g"] == 38.8]
dataframe.iloc[1025584]

code                                                                                                                                                                                                                                                                                                                                              3770013130113
url                                                                                                                                                                                                                                              http://world-en.openfoodfacts.org/product/3770013130113/kefir-de-fruits-gingembre-bio-le-labo-dumoulin-lld-cie
product_name                                                                                                                                                                                                                                                                                            

In [26]:
dataframe["carbohydrates_100g"].sort_values().head(100)

1025584   -1.0
625136     0.0
1424501    0.0
1424502    0.0
1144557    0.0
1424510    0.0
336392     0.0
336390     0.0
336388     0.0
1424511    0.0
336385     0.0
1376432    0.0
680279     0.0
336384     0.0
336383     0.0
1007021    0.0
336382     0.0
336373     0.0
336371     0.0
469951     0.0
1424544    0.0
100039     0.0
1424549    0.0
1424550    0.0
1006941    0.0
336349     0.0
336347     0.0
336341     0.0
469897     0.0
1424499    0.0
1424498    0.0
1424497    0.0
100103     0.0
336462     0.0
336459     0.0
336457     0.0
1564861    0.0
336456     0.0
1659933    0.0
336454     0.0
336451     0.0
336450     0.0
336449     0.0
1659926    0.0
336440     0.0
100087     0.0
336439     0.0
336430     0.0
1007124    0.0
1007120    0.0
469873     0.0
1007117    0.0
1424492    0.0
1424493    0.0
336394     0.0
1424494    0.0
1424495    0.0
100073     0.0
1424496    0.0
336339     0.0
336338     0.0
336334     0.0
336332     0.0
470041     0.0
1006883    0.0
336330     0.0
336328    

determiner les quantile avec scipy pour eliminer les valeur aberrantes
faire un pandas.dataframe.corr pour essayer de trouver une corelation entre deux variable (facteur 1 , variable egal, donc redondant et à sortir)
nettoyer pnns a cause de la casse (librairie nltk)
comparer les sucre et les glucide pour avoir l'indice glycemique

In [22]:
dataframe2 = dataframe.drop(dataframe[dataframe["carbohydrates_100g"] > 100].index)
dataframe2 = dataframe2.drop(dataframe2[dataframe2["carbohydrates_100g"] < 0].index)
dataframe2 = dataframe2.drop(dataframe2[dataframe2["sodium_100g"] > 100].index)
dataframe2 = dataframe2.drop(dataframe2[dataframe2["sodium_100g"] < 0].index)

sns.set_theme(style="whitegrid")
f, ax = plt.subplots(figsize=(6, 15))
dataframe2 = dataframe2.sort_values(by=['carbohydrates_100g'],ascending=False)
sns.set_color_codes("pastel")
sns.barplot(x='carbohydrates_100g', y=dataframe2.index, data=dataframe2,label='carbohydrates_100g', color="b")
ax.legend(ncol=1, loc="lower right", frameon=True,fontsize='large')
ax.xaxis.tick_top()    
# if title == "Study years in selected countries":
#     for i,value in enumerate(dataframe2[sort_by]):
#         ax.text(value+3/xmin, i + 0.2,str(value),fontsize=15)
# else:
#     for i,value in enumerate(dataframe2[sort_by]):
#         ax.text(value+3/xmin, i + 0.2,str(int(value)),fontsize=15)    
plt.xlabel( xlabel="title",fontsize=18)
ax.set(xlim=(0, 100), ylabel="", xlabel='title')
sns.despine(left=True, bottom=True)

In [ ]:

# display(dataframe2)#["carbohydrates_100g"].sort_values(ascending=False).head(1))
sns.set_theme(style="ticks")
# diamonds = sns.load_dataset("diamonds")

f, ax = plt.subplots(figsize=(10, 6))
sns.despine(f)

sns.histplot(
    dataframe2,
    x="carbohydrates_100g",#hue="sodium_100g",
    #multiple="stack",
    palette="light:m_r",
    edgecolor="0.3",
    linewidth=0.5,
#     log_scale=True,
)
ax.xaxis.set_major_formatter(mpl.ticker.ScalarFormatter())
ax.set_xticks([0, 20, 40, 60, 80,100])

In [ ]:
sns.set_theme(style="whitegrid")


# Draw a scatter plot while assigning point colors and sizes to different
# variables in the dataset
f, ax = plt.subplots(figsize=(6.5, 6.5))
sns.despine(f, left=True, bottom=True)
clarity_ranking = ["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"]
sns.scatterplot(x="omega-3-fat_100g", y="omega-9-fat_100g",
                hue="clarity", size="depth",
                palette="ch:r=-.2,d=.3_r",
                hue_order=clarity_ranking,
                sizes=(1, 8), linewidth=0,
                data=dataframe2, ax=ax)